# Rafi Fadhlillah
# 121450143
# Deep Learning RC

# Soal 1 [20 poin]

Tingkatkan performa GRU dengan mengubah fungsi aktivasi dan mengintegrasikan fitur embedding dari pre-trained model seperti Word2Vec.

Tugas:

1. Ubah fungsi aktivasi dari tanh menjadi ReLU.
2. Gunakan embedding dari Word2Vec untuk merepresentasikan data input.

Kunci Analisis: Lakukan perbandingan performa model sebelum dan sesudah modifikasi. Berikan comment anda terhadap akurasinya, di bandingkan yang ada di modul.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from gensim.models import Word2Vec
import json
import warnings
import re
import spacy


nlp = spacy.load('en_core_web_sm')

warnings.filterwarnings('ignore')

# Membaca Dataset
with open('Intent.json', 'rb') as file:
    sentences = json.load(file)

# Lakukan Preprocessing Dataset
def pre_processing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

# Dapatkan judul dari intent

inputs, targets, cls = [], [], []
cls = []
intent_doc = {}

for i in sentences['intents']:
    if i['intent'] not in cls:
        cls.append(i['intent'])

    if i['intent'] not in intent_doc:
        intent_doc[i['intent']] = []

    for text in i['text']:
        inputs.append(pre_processing(text))
        targets.append(i['intent'])

    for response in i['responses']:
        intent_doc[i['intent']].append(response)


In [2]:
# Lanjutkan dengan Tokenizer

def token_data(inp_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')

    tokenizer.fit_on_texts(inp_list)

    inp_seq = tokenizer.texts_to_sequences(inp_list)

    ''' adding padding '''
    inp_seq = tf.keras.preprocessing.sequence.pad_sequences(inp_seq, padding='pre')

    return tokenizer, inp_seq

In [3]:
tokenizer, inp_tensor = token_data(inputs)

In [4]:
def cr_cat_target(targets):
    word = {}
    cat_t = []
    counter=0

    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        cat_t.append(word[trg])

    # Remove the 'dtype' argument
    cat_tensor = tf.keras.utils.to_categorical(cat_t, num_classes=len(word))

    # If you need to ensure the data type, cast it afterwards
    cat_tensor = cat_tensor.astype('int32')

    return cat_tensor, dict((v,k) for k, v in word.items())

In [5]:
target_tensor, target_idx_word = cr_cat_target(targets)

In [6]:
print('input shape: {} and output shape: {}'.format(inp_tensor.shape, target_tensor.shape))

input shape: (143, 9) and output shape: (143, 22)


In [7]:
# Latih Model Word2Vec (Anda mungkin perlu menyesuaikan parameter)
sentences_for_w2v = [text.split() for text in inputs]
w2v_model = Word2Vec(sentences_for_w2v, vector_size=100, window=5, min_count=1, workers=4)

# Buat matriks embedding
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

# Bangun Model GRU dengan aktivasi ReLU dan embedding Word2Vec
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, weights=[embedding_matrix], input_length=inp_tensor.shape[1], trainable=False))
model.add(GRU(128, activation='relu', recurrent_activation='sigmoid'))  # Menggunakan ReLU
model.add(Dense(len(target_idx_word), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Latih model
model.fit(inp_tensor, target_tensor, epochs=10)

# Evaluasi model (Anda bisa menambah metrik evaluasi lainnya)
loss, accuracy = model.evaluate(inp_tensor, target_tensor)
print('Loss:', loss)
print('Akurasi:', accuracy)

# Bandingkan dengan model asli (aktivasi tanh)
# Bangun model asli
original_model = Sequential()
original_model.add(Embedding(len(tokenizer.word_index) + 1, 100, weights=[embedding_matrix], input_length=inp_tensor.shape[1], trainable=False))
original_model.add(GRU(128, activation='tanh', recurrent_activation='sigmoid'))  # Menggunakan tanh
original_model.add(Dense(len(target_idx_word), activation='softmax'))

original_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
original_model.fit(inp_tensor, target_tensor, epochs=10)
loss_original, accuracy_original = original_model.evaluate(inp_tensor, target_tensor)
print('Loss Model Asli:', loss_original)
print('Akurasi Model Asli:', accuracy_original)

# Bandingkan kinerja
print('\nModel dengan aktivasi ReLU:')
print('Loss:', loss)
print('Akurasi:', accuracy)
print('\nModel Asli (tanh):')
print('Loss:', loss_original)
print('Akurasi:', accuracy_original)

# Analisis model mana yang lebih baik (ReLU atau tanh)
if accuracy > accuracy_original:
    print('\nModel dengan fungsi aktivasi ReLU memberikan kinerja yang lebih baik.')
elif accuracy < accuracy_original:
    print('\nModel asli dengan fungsi aktivasi tanh memberikan kinerja yang lebih baik.')
else:
    print('\nKedua model memiliki kinerja yang serupa.')


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.0457 - loss: 3.0917
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0999 - loss: 3.0886
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0663 - loss: 3.0873
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.1044 - loss: 3.0865
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1616 - loss: 3.0835
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.1239 - loss: 3.0823
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1167 - loss: 3.0797
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0909 - loss: 3.0789
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0795 - loss: 3.0766
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1566 - loss: 3.0732
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.1509 - loss: 3.0635
Loss: 3.0714774131774902
Akurasi: 0.10489510744810104
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms

Modifikasi model GRU dengan mengubah fungsi aktivasi dari tanh ke ReLU menghasilkan sedikit peningkatan akurasi, dari 0.0559 menjadi 0.1049. Meskipun fungsi aktivasi ReLU umumnya berkinerja lebih baik untuk masalah vanishing gradient dan ekstraksi fitur yang kompleks, peningkatan akurasi yang kecil ini menunjukkan bahwa perubahan fungsi aktivasi bukanlah satu-satunya faktor kunci dan fitur lain dari model ini mungkin juga memerlukan perubahan untuk melihat peningkatan kinerja yang signifikan.

# No 2

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

# Data dummy (ganti dengan data dari modul)
sentences = [
    "model deep learning memerlukan eksperimen".split(),
    "bidirectional LSTM memproses data sekuensial".split(),
    "kombinasi dengan GRU meningkatkan performa".split(),
]

labels = np.array([0, 1, 1])  # Label dummy

# 1. Word2Vec Embedding
word2vec = Word2Vec(sentences, vector_size=50, min_count=1, workers=4)
vocab_size = len(word2vec.wv)

# Membuat word-index mapping
word_index = {word: i for i, word in enumerate(word2vec.wv.index_to_key)}

# Mengubah data ke bentuk indeks
def sentence_to_indices(sentence):
    return [word_index[word] for word in sentence]

data = np.array([sentence_to_indices(sentence) for sentence in sentences])

# Padding sequences
max_length = max(len(seq) for seq in data)
data_padded = pad_sequences(data, maxlen=max_length, padding="post")

# 2. Model Hibrid: Bidirectional LSTM dan GRU
def build_hybrid_model(hidden_size):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=50, weights=[word2vec.wv.vectors], trainable=False),
        Bidirectional(LSTM(hidden_size, return_sequences=True)),
        GRU(hidden_size, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 3. Eksperimen dengan berbagai hidden layer size
hidden_sizes = [64, 128, 256]
results = {}

for size in hidden_sizes:
    print(f"\nTraining model dengan hidden size: {size}")
    model = build_hybrid_model(size)
    history = model.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)
    loss, accuracy = model.evaluate(data_padded, labels, verbose=0)
    results[size] = accuracy
    print(f"Accuracy dengan hidden size {size}: {accuracy:.4f}")

# 4. Analisis
print("\n--- Analisis ---")
for size, acc in results.items():
    print(f"Hidden size {size}: Accuracy = {acc:.4f}")

best_size = max(results, key=results.get)
print(f"Model dengan hidden size terbaik adalah {best_size} dengan akurasi {results[best_size]:.4f}")



Training model dengan hidden size: 64
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.3889 - loss: 0.6942
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.6893
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6111 - loss: 0.6846
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7778 - loss: 0.6736
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7778 - loss: 0.6630 
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6111 - loss: 0.6722
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6111 - loss: 0.6684 
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7778 - loss: 0.6422
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6111 - loss: 0.6577
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7778 - loss: 0.6225
Accuracy dengan hidden size 64: 0.6667

Training model dengan hidden size: 128
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step 

Accuracy dengan hidden size 256: 1.0000

--- Analisis ---
Hidden size 64: Accuracy = 0.6667
Hidden size 128: Accuracy = 0.6667
Hidden size 256: Accuracy = 1.0000
Model dengan hidden size terbaik adalah 256 dengan akurasi 1.0000


# No 3

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Data dummy (ganti dengan data dari modul)
sentences = [
    "pemahaman konteks dalam data sekuensial sangat penting",
    "bidirectional LSTM membaca data dari dua arah",
    "penggunaan embedding pretrained dapat meningkatkan akurasi",
]

labels = np.array([0, 1, 1])  # Label dummy

# 1. Tokenisasi dan Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
data = tokenizer.texts_to_sequences(sentences)
max_length = max(len(seq) for seq in data)
data_padded = pad_sequences(data, maxlen=max_length, padding="post")

# 2. Memuat GloVe Pre-trained Embedding
embedding_index = {}
glove_file = "glove.6B.50d.txt"  # Pastikan file ini tersedia
with open(glove_file, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

# Membuat matriks embedding
embedding_dim = 50
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# 3. Model dengan Pre-trained GloVe
def build_model_with_glove():
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  weights=[embedding_matrix], trainable=False),
        Bidirectional(LSTM(128, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 4. Model tanpa Pre-trained Embedding
def build_model_without_glove():
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, trainable=True),
        Bidirectional(LSTM(128, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Training dan evaluasi model
print("\n--- Model dengan Pre-trained GloVe ---")
model_with_glove = build_model_with_glove()
model_with_glove.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)
loss_with_glove, acc_with_glove = model_with_glove.evaluate(data_padded, labels, verbose=0)
print(f"Accuracy dengan pre-trained embedding GloVe: {acc_with_glove:.4f}")

print("\n--- Model tanpa Pre-trained Embedding ---")
model_without_glove = build_model_without_glove()
model_without_glove.fit(data_padded, labels, epochs=10, batch_size=2, verbose=1)
loss_without_glove, acc_without_glove = model_without_glove.evaluate(data_padded, labels, verbose=0)
print(f"Accuracy tanpa pre-trained embedding: {acc_without_glove:.4f}")

# Analisis
print("\n--- Analisis ---")
print(f"Accuracy dengan pre-trained GloVe: {acc_with_glove:.4f}")
print(f"Accuracy tanpa pre-trained embedding: {acc_without_glove:.4f}")
if acc_with_glove > acc_without_glove:
    print("Penggunaan pre-trained GloVe memberikan keuntungan dalam akurasi.")
else:
    print("Model tanpa pre-trained embedding memiliki performa serupa atau lebih baik.")

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.50d.txt'